In [1]:
import json
from openai import OpenAI
import jq

In [2]:
# from agentops import Client
# ao_client = Client()

In [3]:
client = OpenAI()

## Get arbitrary data outputs

In [4]:
with open('gcal.json', 'r') as f:
    gcal = json.load(f)
    
with open('outlook.json', 'r') as f:
    outlook = json.load(f)

## Provide the schema for the data you want

Fields I want:
```
{
    "id": "string",
    "start_time": "timestamp",
    "end_time": "timestamp",
    "time_zone": "string",
    "status": "string - confirmed/busy/tentative",
    "location": "string",
    "summary": "string",
    "subject": "string",
    "recurrance": "boolean",
    "visibility":"string"
}
```

In [5]:
desired_schema = {
  "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "properties": {
    "id": {
      "type": "string"
    },
    "start_time": {
      "$ref": "#/$defs/timestamp"
    },
    "end_time": {
      "$ref": "#/$defs/timestamp"
    },
    "time_zone": {
      "type": "string"
    },
    "status": {
      "type": "string",
      "enum": ["confirmed", "busy", "tentative"]
    },
    "location": {
      "type": "string"
    },
    "summary": {
      "type": "string"
    },
    "subject": {
      "type": "string"
    },
    "recurrence": {
      "type": "boolean"
    },
    "visibility": {
      "type": "string"
    }
  },
  "required": ["id", "start_time", "end_time", "status", "subject", "recurrance", "visibility"],
  "$defs": {
    "timestamp": {
      "type": "string",
      "format": "date-time"
    }
  }
}

## Validation layer

In [6]:
def identify_key(key_name, input_schema):

    messages = [{
             "role": "system",
             "content": """You are a perfect system designed to validate and extract data from JSON files. 
For each field, you provide a short check about your reasoning. Go line by line and do a side by side comparison. For example:

"id" | "id" : The field name is identical. True.
"time" | "timestamp": This is the same concept, therefore it counts. True.
"addr" | "Address": This is the same concept and is a , therefore it counts. True.
"cats" | None: There no matching or remotely similar fields. False.

Some fields may not have the exact same names. Use your best judgement about the meaning of the field to determine if they should count.

You come to a definitive conclusion- True or False at the end of your response."""
            }, 
            {
             "role": "user",
             "content": f"Is `{key_name}` present in the desired schema:\n\n  {input_schema}"
            }]

    reasoning_response = client.chat.completions.create(messages=messages,
                                                        model="gpt-4")
    return reasoning_response.choices[0].message.content


In [7]:
identify_key('summary', gcal)

'"summary" | "summary": The field name is identical. True.'

In [8]:
gcal

{'kind': 'calendar#event',
 'etag': '"317317317317317317"',
 'id': '123abc456def',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=123abc456def',
 'created': '2024-02-10T12:00:00Z',
 'updated': '2024-02-10T15:00:00Z',
 'summary': 'Project Launch Meeting',
 'description': 'Initial meeting to discuss the roadmap and deliverables for the new project.',
 'location': 'Conference Room B, 123 Business Rd, City, Country',
 'colorId': '9',
 'creator': {'id': 'creator01',
  'email': 'creator@example.com',
  'displayName': 'Alex Smith',
  'self': False},
 'organizer': {'id': 'org01',
  'email': 'organizer@example.com',
  'displayName': 'Company ABC',
  'self': True},
 'start': {'date': None,
  'dateTime': '2024-02-15T09:00:00Z',
  'timeZone': 'America/New_York'},
 'end': {'date': None,
  'dateTime': '2024-02-15T10:00:00Z',
  'timeZone': 'America/New_York'},
 'endTimeUnspecified': False,
 'recurrence': ['RRULE:FREQ=WEEKLY;COUNT=4'],
 'recurringEventId': '',
 'origin

In [9]:
def to_bool(response: str):
    if 'true' in response.lower():
        return True
    return False

In [10]:
desired_schema['properties']

{'id': {'type': 'string'},
 'start_time': {'$ref': '#/$defs/timestamp'},
 'end_time': {'$ref': '#/$defs/timestamp'},
 'time_zone': {'type': 'string'},
 'status': {'type': 'string', 'enum': ['confirmed', 'busy', 'tentative']},
 'location': {'type': 'string'},
 'summary': {'type': 'string'},
 'subject': {'type': 'string'},
 'recurrence': {'type': 'boolean'},
 'visibility': {'type': 'string'}}

In [11]:
to_bool('"summary" | "summary" : The field name is identical.')

False

In [12]:
to_bool('"summary" | "summary": The field name is identical. True.\n\nBased on the comparison, the field "summary" is present in the desired schema.')

True

In [13]:
gcal

{'kind': 'calendar#event',
 'etag': '"317317317317317317"',
 'id': '123abc456def',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=123abc456def',
 'created': '2024-02-10T12:00:00Z',
 'updated': '2024-02-10T15:00:00Z',
 'summary': 'Project Launch Meeting',
 'description': 'Initial meeting to discuss the roadmap and deliverables for the new project.',
 'location': 'Conference Room B, 123 Business Rd, City, Country',
 'colorId': '9',
 'creator': {'id': 'creator01',
  'email': 'creator@example.com',
  'displayName': 'Alex Smith',
  'self': False},
 'organizer': {'id': 'org01',
  'email': 'organizer@example.com',
  'displayName': 'Company ABC',
  'self': True},
 'start': {'date': None,
  'dateTime': '2024-02-15T09:00:00Z',
  'timeZone': 'America/New_York'},
 'end': {'date': None,
  'dateTime': '2024-02-15T10:00:00Z',
  'timeZone': 'America/New_York'},
 'endTimeUnspecified': False,
 'recurrence': ['RRULE:FREQ=WEEKLY;COUNT=4'],
 'recurringEventId': '',
 'origin

In [14]:
for key in desired_schema['properties'].keys():
    response = identify_key(key, gcal)
    print(response)
    if to_bool(response):
        line=f"✅ {key}"
    else:
        line = f"❌ {key}"
    if key in desired_schema['required']:
        line = "！"+line
        
    print(line)

"id" | "id" : The field name is identical. True.
！✅ id
"start_time" | "start": The field label "start_time" in the given field does not directly match any field name in the JSON schema. However, the JSON schema includes a "start" object that contains a "dateTime" field, which could represent the start time of an event. Therefore, it can be somewhat inferred that "start" corresponds to "start_time" but they are not identical. Therefore, False.
！❌ start_time
"end_time" | "end": The field name is not identical, but they represent the same concept, that is, the end timing of an event. True. 

So, for the given JSON schema, the conclusion is True. The 'end_time' can be considered to be present as the 'end' field.
！✅ end_time
"time_zone" | {"start": {'timeZone': 'America/New_York'}, "end": {'timeZone': 'America/New_York'}, "originalStartTime": {'timeZone': 'America/New_York'}}:  
The fieldType 'timeZone' is nested inside the fields 'start', 'end' and 'originalStartTime'. Therefore it counts.

### 2. Devise a way to pull the fields from the data we have

**Testing it out...**

In [17]:
desired_schema

{'$schema': 'http://json-schema.org/draft-07/schema#',
 'type': 'object',
 'properties': {'id': {'type': 'string'},
  'start_time': {'$ref': '#/$defs/timestamp'},
  'end_time': {'$ref': '#/$defs/timestamp'},
  'time_zone': {'type': 'string'},
  'status': {'type': 'string', 'enum': ['confirmed', 'busy', 'tentative']},
  'location': {'type': 'string'},
  'summary': {'type': 'string'},
  'subject': {'type': 'string'},
  'recurrence': {'type': 'boolean'},
  'visibility': {'type': 'string'}},
 'required': ['id',
  'start_time',
  'end_time',
  'status',
  'subject',
  'recurrance',
  'visibility'],
 '$defs': {'timestamp': {'type': 'string', 'format': 'date-time'}}}

In [18]:
def create_jq_string(input_schema, key):
    messages = [{
                 "role": "system",
                 "content": f"""You are a perfect jq engineer designed to validate and extract data from JSON files using jq. Only reply with code. Do NOT use any natural language. Do NOT use markdown i.e. ```.
                 
Your task is to create a jq filter to extract the data from the following JSON:

{input_schema}
"""
                }, 
                {
                 "role": "user",
                 "content": f"Write jq to extract the key {key}"
                }]

    response = client.chat.completions.create(messages=messages, model="gpt-4-0125-preview")
    return response.choices[0].message.content

In [19]:
jq_reasoning = create_jq_string(gcal, 'time')

In [20]:
print(jq_reasoning)

.time_zone


## Condense into a single filter

## Try jq

In [21]:
def repair_query(query, error, input_schema):
    messages = [{
                "role": "system",
                "content": "You are a perfect jq engineer designed to validate and extract data from JSON files using jq. Only reply with code. Do NOT use any natural language. Do NOT use markdown i.e. ```."
                },
                {
                "role": "user",
                "content": f"""The following query returned an error while extracting from the follwing schema:
                
Query: {query}

Error: {error}

Schema: {input_schema}"""}]
    response = client.chat.completions.create(messages=messages,
                                              model="gpt-4-0125-preview")
    return response.choices[0].message.content    
    
    

In [53]:
filter_query = {}

for key in list(desired_schema['properties'].keys()):
#     print(key)
    
    while True:
        jq_string = create_jq_string(gcal, key)
        print(jq_string)
        query = ""
        try:
            query = jq.compile(jq_string).input(gcal).all()
            break
        except Exception as e:
#             print(e)
            repair_query(query, str(e), gcal)
    print(query)
    filter_query[key] = jq_string

.id
['123abc456def']
.start.dateTime
['2024-02-15T09:00:00Z']
.end.dateTime
['2024-02-15T10:00:00Z']
.kind, .etag, .id, .status, .htmlLink, .created, .updated, .summary, .description, .location, .colorId, .creator, .organizer, .start.timeZone, .end.timeZone, .endTimeUnspecified, .recurrence[], .recurringEventId, .originalStartTime.timeZone, .transparency, .visibility, .iCalUID, .sequence, .attendees[].email, .attendeesOmitted, .extendedProperties.private, .extendedProperties.shared, .hangoutLink, .conferenceData, .gadget, .anyoneCanAddSelf, .guestsCanInviteOthers, .guestsCanModify, .guestsCanSeeOtherGuests, .privateCopy, .locked, .reminders, .source, .workingLocationProperties, .outOfOfficeProperties, .focusTimeProperties, .attachments, .eventType
['calendar#event', '"317317317317317317"', '123abc456def', 'confirmed', 'https://www.google.com/calendar/event?eid=123abc456def', '2024-02-10T12:00:00Z', '2024-02-10T15:00:00Z', 'Project Launch Meeting', 'Initial meeting to discuss the roadma

In [60]:
json.dumps(filter_query)

'{"id": ".id", "start_time": ".start.dateTime", "end_time": ".end.dateTime", "time_zone": ".kind, .etag, .id, .status, .htmlLink, .created, .updated, .summary, .description, .location, .colorId, .creator, .organizer, .start.timeZone, .end.timeZone, .endTimeUnspecified, .recurrence[], .recurringEventId, .originalStartTime.timeZone, .transparency, .visibility, .iCalUID, .sequence, .attendees[].email, .attendeesOmitted, .extendedProperties.private, .extendedProperties.shared, .hangoutLink, .conferenceData, .gadget, .anyoneCanAddSelf, .guestsCanInviteOthers, .guestsCanModify, .guestsCanSeeOtherGuests, .privateCopy, .locked, .reminders, .source, .workingLocationProperties, .outOfOfficeProperties, .focusTimeProperties, .attachments, .eventType", "status": ".status", "location": ".location", "summary": ".summary", "subject": ".kind", "recurrence": ".recurrence", "visibility": ".visibility"}'

In [85]:
def dict_to_jq_filter(transformation_dict):
    jq_filter_parts = []
    for new_key, json_path in transformation_dict.items():
        # For each item in the dictionary, create a string '"new_key": json_path'
        # Note: json_path is assumed to be a valid jq path expression as a string
        jq_filter_parts.append(f'"{new_key}": {json_path}')
    
    # Join all parts with commas and wrap in braces to form a valid jq object filter
    jq_filter = "{ " + ",\n ".join(jq_filter_parts) + " }"
    return jq_filter

# Example usage
# del filter_query['time_zone']
jq_query = dict_to_jq_filter(filter_query)
print(jq_query)


{ "id": .id,
 "start_time": .start.dateTime,
 "end_time": .end.dateTime,
 "status": .status,
 "location": .location,
 "summary": .summary,
 "subject": .kind,
 "recurrence": .recurrence,
 "visibility": .visibility }


In [86]:
# jq_query = '''
# {
#   "id": .id
# }
# '''

# Apply the jq filter
result = jq.compile(jq_query).input(gcal).all()

print(result)

[{'id': '123abc456def', 'start_time': '2024-02-15T09:00:00Z', 'end_time': '2024-02-15T10:00:00Z', 'status': 'confirmed', 'location': 'Conference Room B, 123 Business Rd, City, Country', 'summary': 'Project Launch Meeting', 'subject': 'calendar#event', 'recurrence': ['RRULE:FREQ=WEEKLY;COUNT=4'], 'visibility': 'public'}]


In [34]:
print(condensed_response)

{
  "kind": "calendar#event",
  "etag": "\"317317317317317317\"",
  "id": .id,
  "status": .status,
  "htmlLink": ("https://www.google.com/calendar/event?eid=" + .id),
  "created": .start_time,
  "updated": .end_time,
  "summary": if .summary then .summary else .subject end,
  "description": "Initial meeting to discuss the roadmap and deliverables for the new project.",
  "location": .location,
  "colorId": "9",
  "creator": {
    "id": "creator01",
    "email": "creator@example.com",
    "displayName": "Alex Smith",
    "self": false
  },
  "organizer": {
    "id": "org01",
    "email": "organizer@example.com",
    "displayName": "Company ABC",
    "self": true
  },
  "start": {
    "dateTime": .start_time,
    "timeZone": .time_zone
  },
  "end": {
    "dateTime": .end_time,
    "timeZone": .time_zone
  },
  "endTimeUnspecified": false,
  "recurrence": ["RRULE:FREQ=WEEKLY;COUNT=4"],
  "recurringEventId": "",
  "originalStartTime": {
    "dateTime": .start_time,
    "timeZone": .time_